In [56]:
print('all ok')

all ok


In [57]:
%pwd

'c:\\Users\\Bhavj\\Documents\\Healthwise-ChatBot'

In [58]:
import os
os.chdir('../')

In [59]:
%pwd

'c:\\Users\\Bhavj\\Documents'

In [65]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [91]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [92]:
extracted_data = load_pdf_files("data")

FileNotFoundError: Directory not found: 'data'

In [93]:
len(extracted_data)

637

In [94]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Documents objects, return a new list of document objects,
    containing only 'source' in metadeta and the original page content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get('source')
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={'source': src}
            )
        )
    return minimal_docs

In [95]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [96]:
minimal_docs[0].metadata

{'source': 'data\\Medical_book.pdf'}

In [97]:
# split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [98]:
texts_chunk=text_split(minimal_docs)
print(f'Number of Chunks: {len(texts_chunk)}')


Number of Chunks: 5859


## Embeddings


In [99]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download the embeddings model from HuggingFace.
    """
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
    )
    return embeddings

embeddings = download_embeddings()

In [100]:
vector = embeddings.embed_query('hello world')

In [101]:
vector

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [102]:
print('Vector length:',len(vector))

Vector length: 384


In [103]:
from dotenv import load_dotenv
import os
load_dotenv()

False

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [ ]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
pc

In [ ]:
from pinecone import Pinecone, ServerlessSpec

index_name = "healthwise-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    
index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embeddings,
    index_name=index_name
    
)

In [ ]:
# load existing index
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

## Add more data to the existing Pinecone index


In [ ]:
bhavj=Document(
    page_content="This is a test document where i provide machine learning and rag tutorials.",
    metadata={"source": "test_source.pdf"}
)

In [ ]:
docsearch.add_documents(documents=[bhavj])

['bdb5dd8b-103c-429b-a624-dff8dd30a5e5']

In [ ]:
retriever =  docsearch.as_retriever(search_type='similarity', search_kwargs={"k": 3})

In [ ]:
retrieved_docs=retriever.invoke('Example of chronic obstructive pulmonary disease?')
retrieved_docs

[Document(id='41cafdd4-ecce-4b81-982a-c6a13d50169f', metadata={'source': 'data\\Medical_book.pdf'}, page_content='slow onset over a long period of time.\nChronic obstructive pulmonary disease (COPD)—\nA term used to describe chronic lung diseases, like\nchronic bronchitis, emphysema, and asthma.\nEmphysema —One of the several diseases called\nchronic obstructive pulmonary diseases, emphyse-\nma involves the destruction of air sac walls to form\nabnormally large air sacs that have reduced gas\nexchange ability and that tend to retain air within\nthe lungs. Symptoms include labored breathing,'),
 Document(id='9ee2d4cf-e997-483a-b227-828ce0a9e2c1', metadata={'source': 'data\\Medical_book.pdf'}, page_content='men than women developed chronic bronchitis, but as\nthe number of women who smoke has increased, so has\ntheir rate of chronic bronchitis. Because this disease pro-\ngresses slowly, middle-aged and older people are more\nlikely to be diagnosed with chronic bronchitis.\nChronic bronch

In [ ]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model='gpt-3.5-turbo')

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"

)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({'input':"What is acromegaly and gigantism?"})
print(response['answer'])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}